In [ ]:
import rlrom.utils as rlu
import polars as pl

In [ ]:
from rlrom.testers import RLTester

cfg_file= 'cfg0tr_ppo_specs.yml'
cfg = rlu.load_cfg(cfg_file)


# Collect existing training folders and load results

In [ ]:
cfg_file= 'cfg0tr_ppo_specs.yml'
cfg = rlu.load_cfg(cfg_file)
df_all_training_files = rlu.get_df_all_training_files(cfg)
print(df_all_training_files.collect())

Show the path for the first one. 

In [ ]:
print(df_all_training_files.collect()['path'][0])

To get the complete list of strings: 

In [ ]:
print(df_all_training_files.collect()['path'].__array__())

In [ ]:
df_training_files6 = df_all_training_files.collect().filter(pl.col('num')==6)['training_files'][0]
print(df_training_files6.collect())

## Load file results

In [ ]:
df_all_res = rlu.get_df_load_all_training_res(df_all_training_files)


In [ ]:
print(df_all_res.collect().head())
print(df_all_res.collect().tail())


# Plotting with Matplotlib

In [ ]:
import rlrom.plots as rlp

df_all_res has all training data, with fields from res_all_ep expanded. 

In [ ]:
df_enveloppe = rlu.get_df_mean_min_max_val(df_all_res, 'mean_ep_rew')

In [ ]:
ax = rlp.plot_df_enveloppe(df_all_res, 'mean_ep_rew', label='Mean episode reward', linestyle='--', color='blue')
ax = rlp.plot_df_enveloppe(df_all_res, 'mean_ep_len', label='Mean episode length', linestyle='--', color='green', ax=ax)


For a formula, we need to unnest the result into init_rob and ratio_sat

In [ ]:
phi = 'phi_left_goal'
df_res_phi = df_all_res.select('label', #training id 
                                    'steps',
                                    phi).unnest(phi)
print(df_res_phi.collect().head())

In [ ]:
ax = rlp.plot_df_enveloppe(df_res_phi, 'ratio_init_sat', label='Ratio Sat of '+phi, linestyle='--', color='blue')

In [ ]:
import os, copy
from rlrom.testers import RLTester

In [ ]:
find_max = pl.col('mean_ep_rew')==pl.col('mean_ep_rew').max()
df_res_max = df_all_res.filter(find_max)

In [ ]:
print(df_res_max.collect().select('steps', 'label', 'path', 'mean_ep_rew'))

# Re-test

## Find and retest best model

In [ ]:
best_model_path = df_res_max.collect()['path'][0]
best_step = df_res_max.collect()['steps'][0]
best_model_file = os.path.join(best_model_path, f'model_step_{str(best_step)}.zip')
print(best_model_file)

In [ ]:
cfg_test = cfg['cfg_test']
cfg_test['model_file']= best_model_file
cfg_test['res_file'] = './best_test_result.yml'
cfg_tmp = copy.deepcopy(cfg)
cfg_tmp = rlu.set_rec_cfg_field(cfg_tmp,cfg_test=cfg_test)
cfg_tmp = rlu.set_rec_cfg_field(cfg_tmp,render_mode=None, num_ep=100)
T = RLTester(cfg_tmp)
Tres = T.run_cfg_test()


## Retest all checkpoints

In [ ]:
import copy,os

def retest_models(df_training_files, num_ep):
    cfg_test = cfg['cfg_test']
    for r in df_training_files.collect().iter_rows(named=True):
        print(f'Retesting for step {r['steps']}, i.e., model {r['res_files']}')
        cfg_test['model_file']= os.path.join(r['path'],r['model_files'])
        cfg_test['res_file'] = r['res_files']        
        cfg_tmp = copy.deepcopy(cfg)  
        cfg_tmp = rlu.set_rec_cfg_field(cfg_tmp,cfg_test=cfg_test)
        cfg_tmp = rlu.set_rec_cfg_field(cfg_tmp,render_mode=None, num_ep=num_ep)
        T = RLTester(cfg_tmp)
        T.run_cfg_test()        
        

In [ ]:
retest_models(df_training_files6,100)

In [ ]:
df_res_retest = rlu.get_df_load_training_res(df_training_files6)

In [ ]:
print(df_res_retest.collect().select('steps', 'mean_ep_rew'))


In [ ]:
df_phi = df_res_retest.select('steps', 'phi_left_goal').unnest('phi_left_goal')
print(df_phi.collect())

# Plotting with plotly (Work in progress, interactive plots)

In [ ]:
import plotly.express as px

## One training,  episode lengths and rewards

In [ ]:

df0 = df_all_res.select('steps', 'mean_ep_len', 'mean_ep_rew', 'label').filter(pl.col('label')=='Training0').collect()

In [ ]:
fig = px.line(df0, x='steps', y = ['mean_ep_len', 'mean_ep_rew'])
fig.show()

## All training episode lens and rewards

In [ ]:
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=1,  subplot_titles=['Mean Episode Length', 'Mean Episode Reward'])

go1 = px.line(df_all_res.collect(), x='steps', y = 'mean_ep_len', color='label')
go2 = px.line(df_all_res.collect(), x='steps', y = 'mean_ep_rew', color='label')

fig.add_traces(go1.data, rows=1, cols=1)
fig.add_traces(go2.data, rows=2, cols=1)

fig.update_layout(height=600, title_text="Training Metrics")
fig.show()

## Adding specs

In [ ]:
phi = 'phi_left_goal'
df_phi = df_all_res.select('label','steps',phi).unnest(phi)

In [ ]:
fig=  px.line(df_phi.collect(), x='steps', y = 'ratio_init_sat', color='label')
fig.show()

In [ ]:
expr_min = lambda feature: pl.col(feature).list.min().name.suffix('_min')
expr_max = lambda feature: pl.col(feature).list.max().name.suffix('_max')
expr_mean = lambda feature: pl.col(feature).list.mean().name.suffix('_mean')

feature = 'mean_ep_len'
df = df_all_res.select('label','steps',feature)
df_enveloppe = df.group_by(pl.col('steps')).agg(pl.col(feature)
                        ).sort(pl.col('steps'))                        


print(df_group.collect().select(expr_min(feature), expr_mean(feature),expr_max(feature)))


In [ ]:
fig = px.line(df_phi.collect(), x='steps', y = 'ratio_init_sat', color='label')
fig.show()